# Pre-process the Retrieved CSV Files from NWM Retrospective NetCDF Files

This jupyter notebook uses results from NWM_RRv2_Retrieve.py (i.e., zipped CSV files) to create single CSV for specified variables. For example, it reads NWM_2007_ALBEDO.zip, NWM_2008_ALBEDO.zip, ... and ouptuts NWM_ALBEDO.csv. Then, it will filter the complete dataset using a specified period and pattern (structure) to create a dataframe. This is becasue I want to have the same time period for NWM, SNOTEL, and SNODAS comparision.

## 1.  Import Libraries

In [2]:
# Import libraries
import os
import glob
import zipfile
import pandas as pd 
import numpy as np 

## 2.  Define Paths

In [3]:
data_dir = '../output'                # The path including zipped cvs files
output_dir = data_dir                 # The path to save final csv files

## 3.  Manipulate NWM Datasets

Unzip NWM results and create a single CSV file for each variable.

In [5]:
for f in glob.glob(os.path.join(data_dir, "*.zip")):
    
    with zipfile.ZipFile(f, 'r') as ref:
        
        ref.extractall(output_dir)

For each variable, read all csv files and create a combined csv file.

In [19]:
variables = ['ALBEDO', 'ALBSND', 'ALBSNI', 'COSZ', 'FIRA', 
             'FSA', 'FSNO', 'HFX', 'LH', 'SNEQV', 'SNOWH', 'TRAD']

for v in variables:
    
    all_filenames = [i for i in glob.glob(os.path.join(data_dir, f'NWM_*_*_{v}.csv'))]
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
    combined_csv.to_csv(os.path.join(output_dir, f'NWM_{v}.csv'), index=False, encoding='utf-8-sig')

## 4.  Filter Dataset for a Specific Time Period

* ### Read Datasets

In [127]:
snotel_info = pd.read_csv(os.path.join(output_dir, 'SNOTEL_indices_at_NWM.csv'))

# put the data that you received from NCAR in output_dir and name it NWM_P.csv
nwm_p = pd.read_csv(os.path.join(output_dir, 'NWM_P.csv'))                       

for v in variables:
    vars()[f'nwm_{v}'] = pd.read_csv(os.path.join(output_dir, f'NWM_{v}.csv'))


* ### Define a Period

In [11]:
dates_Hourly = pd.date_range('2007-01-01', '2007-04-01', freq='1H')

* ### Define Filter Function

The following function loops over all sites and for each day in the dates_Hourly list, it extracts the values of snow water equivalent (LDASOUT outputs) or precipitation (FORCING iputs)  as well as some other information related to the gage. Finally, it returns a dataframe including 4 columns (Site_ID, Ecoregion_Name, Date_Time_UTC, and variale of interest) as the output. 

Note that if precipitation is the varibale of interest, some pre-processing should be done for inputs to the Filter function becasue precipitation values are note retrieved using developed scripts. I asked for them from NWM support team at NCAR. 

In [141]:
def Filter(dataset1, dataset2, column_code, column_val, final_column_val, csv_name, output_dir):
    
    '''
    dataset1:          Dataset for which Filter function is used  
    dataset2:          Dataset including snotel information
    column_code:       Column including station id
    column_val:        Column including values of the variable of interest
    final_column_val:  Column including values of the variable of interests as results 
    csv_name:          Name of the output (i.e., a CSV file)
    output_dir:        Path to save outputs
    '''
    
    # Get station ids
#     code = set(dataset1[column_code])  # List of site codes 
    code = [301, 1000]
    
    ID = []
    NAME = []
    TIME = []
    VALUE = []
    for c in code:
        select = dataset1.loc[dataset1[column_code] == c]
        for d in dates_Hourly:
            try:
                id = select[pd.to_datetime(select['time']) == d][column_code].values[0]
                name = dataset2[dataset2[column_code] == c]['Ecoregion_NAME'].values[0]
                value = select[pd.to_datetime(select['time']) == d][column_val].values[0]
            except Exception as e:
                value = np.nan
            ID.append(id)
            NAME.append(name)
            TIME.append(d)
            VALUE.append(value)

    df = pd.DataFrame({'col1': ID, 'col2': NAME, 'col3': TIME, 'col4': VALUE})
    df.columns = ['Site_Code',  'Ecoregion_Name', 'Date_Time_UTC', final_column_val] 
    df.index = df['Date_Time_UTC']
    df.to_csv(os.path.join(output_dir, csv_name))

* ### Apply Filter Function

In [144]:
# Variables in LDASOUT

for v in variables:
    
    dataset1 = vars()[f'nwm_{v}']         
    dataset2 = vars()['snotel_info'] 
    column_code = 'Station_ID'
    column_val = dataset1.columns[4]
    final_column_val = dataset1.columns[4]
    csv_name = f'NWM_{v}_Filter.csv'

    Filter(dataset1, dataset2, column_code, column_val, final_column_val, csv_name, output_dir)

In [142]:
# Precipitation

dataset1 = vars()['nwm_p']       
dataset1.rename(columns={'site_id':'Station_ID'}, inplace=True)                   # change the column name to be consistent with snotel_info
dataset1['time'] = pd.to_datetime(dataset1['time'], format='%Y%m%d%H')            # make 'time' column as a data/time format 
dataset2 = vars()['snotel_info'] 
column_code = 'Station_ID'
column_val = 'P'
final_column_val = 'P_mm'
csv_name = 'NWM_P_Filter.csv'

Filter(dataset1, dataset2, column_code, column_val, final_column_val, csv_name, output_dir)